In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Age-Related_Macular_Degeneration/GSE29801'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Systems-level analysis of age-related macular degeneration reveals global and subtype-specific functional pathways"
!Series_summary	"Age-related macular degeneration (AMD) is a leading cause of human blindness in developed countries. While significant inroads have been made over the past decade, an integrated description of the molecular mechanisms underlying AMD has yet to emerge. Here we describe a systems-level transcriptome analysis of the retina and retinal pigmented epithelium (RPE)-choroid complex from 31 normal, 26 AMD, and 11 potential pre-AMD human eyes derived from the University of Iowa. Our analysis identifies cell-mediated immune responses as the central feature of dry AMD, wet AMD, and geographic atrophy (GA), and we confirm this finding using a second cohort of donor eyes obtained from the Lion's Eye Bank of Oregon. In addition, in the RPE-choroid, we identify the major up-regulated pathways in GA and wet AMD as apoptosis and angio

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # As the data involves transcriptome analysis, it is likely to contain gene expression data.

# Identifying the keys for 'Age-Related_Macular_Degeneration', 'age', and 'gender'
trait_row = 3  # Based on the given dictionary, 'ocular disease' seems to reflect 'Age-Related_Macular_Degeneration' status.
age_row = 2  # 'age (years)' key reflects the age variable.
gender_row = 1  # 'gender' key directly corresponds to the gender variable.

# Define conversion functions
def convert_trait(value):
    try:
        return 1 if 'AMD' in value.split(': ')[1] else 0
    except:
        return None

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except:
        return None

def convert_gender(value):
    try:
        return 1 if 'male' in value.split(': ')[1] else 0
    except:
        return None

# Save cohort information
save_cohort_info('GSE29801', './preprocessed/Age-Related_Macular_Degeneration/cohort_info.json', is_gene_available, trait_row is not None)

# Extract and process clinical feature data
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Age-Related_Macular_Degeneration', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Age-Related_Macular_Degeneration/trait_data/GSE29801.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM738433': [0.0, 9.0, 1.0], 'GSM738434': [0.0, 9.0, 1.0], 'GSM738435': [0.0, 10.0, 1.0], 'GSM738436': [0.0, 10.0, 1.0], 'GSM738437': [0.0, 18.0, 1.0], 'GSM738438': [0.0, 18.0, 1.0], 'GSM738439': [0.0, 21.0, 1.0], 'GSM738440': [0.0, 21.0, 1.0], 'GSM738441': [0.0, 34.0, 1.0], 'GSM738442': [0.0, 34.0, 1.0], 'GSM738443': [0.0, 36.0, 1.0], 'GSM738444': [0.0, 36.0, 1.0], 'GSM738445': [0.0, 37.0, 1.0], 'GSM738446': [0.0, 37.0, 1.0], 'GSM738447': [0.0, 40.0, 1.0], 'GSM738448': [0.0, 40.0, 1.0], 'GSM738449': [0.0, 44.0, 1.0], 'GSM738450': [0.0, 45.0, 1.0], 'GSM738451': [0.0, 45.0, 1.0], 'GSM738452': [0.0, 47.0, 1.0], 'GSM738453': [0.0, 48.0, 1.0], 'GSM738454': [0.0, 48.0, 1.0], 'GSM738455': [0.0, 48.0, 1.0], 'GSM738456': [0.0, 48.0, 1.0], 'GSM738457': [0.0, 49.0, 1.0], 'GSM738458': [0.0, 49.0, 1.0], 'GSM738459': [0.0, 49.0, 1.0], 'GSM738460': [0.0, 49.0, 1.0], 'GSM738461': [0.0, 49.0, 1.0], 'GSM738462': [0.0, 49.0, 1.0], 'GSM738463': [0.0, 55.0, 1.0], 'GSM738464': [0.0, 61.0, 1.0], 'GSM73846

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '26', '27', '28', '29', '30', '31', '32'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['266', '266', '266', '266', '266'], 'ROW': [170.0, 168.0, 166.0, 164.0, 162.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan], 'SPOT_ID.1': [nan, nan, nan, nan, nan], 'ORDER': [1.0, 2.0, 3.

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Age-Related_Macular_Degeneration/gene_data/GSE29801.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Age-Related_Macular_Degeneration')

# 4. Save the cohort information.
save_cohort_info('GSE29801', './preprocessed/Age-Related_Macular_Degeneration/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Age-Related_Macular_Degeneration/GSE29801.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Age-Related_Macular_Degeneration', the least common label is '1.0' with 142 occurrences. This represents 48.46% of the dataset.
The distribution of the feature 'Age-Related_Macular_Degeneration' in this dataset is fine.

Quartiles for 'Age':
  25%: 64.0
  50% (Median): 78.0
  75%: 86.0
Min: 9.0
Max: 101.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 293 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.

